## Setup and Import Libraries

In [2]:
import time
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm, trange
from utils import Utils

import warnings
warnings.filterwarnings('ignore')

In [3]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()
OPENAI_API_KEY = utils.get_openai_api_key()

### Load the Dataset

!wget -q --show-progress -O all-the-news-3.zip "https://www.dropbox.com/scl/fi/wruzj2bwyg743d0jzd7ku/all-the-news-3.zip?rlkey=rgwtwpeznbdadpv3f01sznwxa&dl=1"

!unzip all-the-news-3.zip

In [4]:
with open('data/all-the-news-3.csv', 'r') as f:
    header = f.readline()
    print(header)

date,year,month,day,author,title,article,url,section,publication



In [5]:
df = pd.read_csv('data/all-the-news-3.csv', nrows=99)
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


## Setup Pinecone

In [6]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

INDEX_NAME = utils.create_index_name('t-ind')

pinecone = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [7]:
INDEX_NAME

't-ind-thpugi5b8qynkab8tfzq0ae9gcrijw7ctqaa'

## Create Embeddings of the News Titles

In [8]:
def get_embeddings(articles, model="text-embedding-ada-002"):
    return openai_client.embeddings.create(input = articles, model=model)

In [9]:
CHUNK_SIZE=400
TOTAL_ROWS=10000

progress_bar = tqdm(total=TOTAL_ROWS)
chunks = pd.read_csv('data/all-the-news-3.csv', chunksize=CHUNK_SIZE, nrows=TOTAL_ROWS)

chunk_num = 0
for chunk in chunks:
    titles = chunk['title'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(chunk_num*CHUNK_SIZE+i), 'values':embeddings.data[i].embedding,
                'metadata':{'title':titles[i]},} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        index.upsert(prepped)
        prepped = []
    progress_bar.update(len(chunk))

100%|██████████| 10000/10000 [04:01<00:00, 26.65it/s]

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

100%|██████████| 10000/10000 [04:20<00:00, 26.65it/s]

### Build the Recommender System

In [11]:
def get_recommendations(pinecone_index, search_term, top_k=10):
    embed = get_embeddings([search_term]).data[0].embedding
    res = pinecone_index.query(vector=embed, top_k=top_k, include_metadata=True)
    return res

In [12]:
reco = get_recommendations(index, 'obama')

for r in reco.matches:
    print(f'{r.score} : {r.metadata["title"]}')

0.848942161 : Barack Obama just stepped off the sidelines to defend Obamacare
0.847827375 : President Obama has a new plan to fight the opioid epidemic
0.847774625 : “Our democracy is at stake”: Obama delivers his first post-presidency campaign speech
0.847268581 : Obama: if you were fine with big government until it served black people, rethink your biases
0.846126914 : President Obama: Michelle & I Are Gonna Be Renters
0.843110383 : Vox Sentences: Obama got a warmer welcome in Hiroshima than the Japanese prime minister
0.84294188 : Obama meets with national security team on Syria, Islamic State
0.842331529 : Watch President Obama dance the tango in Argentina
0.840474308 : Barack Obama in talks to create shows for Netflix: New York Times
0.839662194 : Obama and Supreme Court Tag Team on Juvenile Justice Reform


## Create Embeddings of All News Content

In [13]:
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(name=INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

articles_index = pinecone.Index(INDEX_NAME)

In [14]:
def embed(embeddings, title, prepped, embed_num):
    for embedding in embeddings.data:
        prepped.append({'id':str(embed_num), 'values':embedding.embedding, 'metadata':{'title':title}})
        embed_num += 1
        if len(prepped) >= 100:
            articles_index.upsert(prepped)
            prepped.clear()
    return embed_num

In [15]:
news_data_rows_num = 100

embed_num = 0 #keep track of embedding number for 'id'
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)

prepped = []
df = pd.read_csv('data/all-the-news-3.csv', nrows=news_data_rows_num)
articles_list = df['article'].tolist()
titles_list = df['title'].tolist()

for i in range(0, len(articles_list)):
    print(".",end="")
    art = articles_list[i]
    title = titles_list[i]
    if art is not None and isinstance(art, str):
        texts = text_splitter.split_text(art)
        embeddings = get_embeddings(texts)
        embed_num = embed(embeddings, title, prepped, embed_num)

....................................................................................................

In [16]:
articles_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

### Build the Recommender System

In [17]:
reco = get_recommendations(articles_index, 'obama', top_k=100)
seen = {}

for r in reco.matches:
    title = r.metadata['title']
    if title not in seen:
        print(f'{r.score} : {title}')
        seen[title] = '.'

0.821350217 : Why Obama is vetting Nevada's Republican governor for the Supreme Court
0.818117857 : U.S. lawmakers ask for disclosure of number of Americans under surveillance
0.811663449 : NYPD Honcho Insulted by 'Hamilton' Star Lin-Manuel Miranda Celebrating Obama's Controversial Prisoner Release
0.805786252 : Why Jews Are Getting Themselves Arrested at ICE Centers Around the Country
0.805666924 : Trump keeping options open as Republican feud rages
0.800964475 : Michael Bloomberg Is Seriously Considering a Presidential Run
0.798702419 : The most revealing Republican ad of the election is an attack ad against Tim Kaine
0.798462033 : Exclusive: Trump considering fracking mogul Harold Hamm as energy secretary - sources
0.797424436 : Trump tells anti-abortion marchers he will support them
0.79741168 : The government official in charge of ethics just harshly condemned Trump’s plan
0.793310285 : Exclusive: China shuns U.S. request for talks on airline website dispute over Taiwan
0.79156917